In [1]:
# ! pip install fastparquet
# ! pip install ipynb

In [42]:
from ipynb.fs.full.data_merge_utils import reduce_mem_usage
from ipynb.fs.full.data_merge_utils import merge_data


In [3]:
import pandas as pd
import swifter
import numpy as np
from glob import glob
import json
import gc


In [4]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [5]:
attribute_df = pd.read_parquet("../data_phase1/attributes.parquet")

In [6]:
train = train.merge(attribute_df, on='product_id', how='left')
val = val.merge(attribute_df, on='product_id', how='left')


In [7]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,642878c18e40a1028b6610cd3396d4ee4fb6ad9623f5bb...,a37fdccd28723fba01a2f0a68d3bb1a3b847e550a7b0c2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,206b88b91a241cc6ed5af856ce3ddda4dccf91ae2ec798...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,570.041667,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,28d2b1e7b0970e2c58966c47b4c7ab7426e43a8689007e...
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,f6bafdefb3c04b392adc1d139ea55a565c8829627bb7d3...,884c59e2814e352d318a8447c2f32a0370955af64838e3...,393b930ad3c40c6af1c14ce8572f39a3a2027ef70dd21e...,de60eee3c057eab83a9987275b3ecfb080026c3297b036...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,208.041667,c4aebf95aa0044c5230c104faa9ef2b5ccfab836b764d4...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [8]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


Memory usage of dataframe is 802.92 MB
Memory usage after optimization is: 475.06 MB
Decreased by 40.8%
Memory usage of dataframe is 152.04 MB
Memory usage after optimization is: 146.25 MB
Decreased by 3.8%


In [9]:
gc.collect()

521

In [10]:
train.shape, val.shape

((3507990, 29), (687192, 28))

In [11]:
train['days_elapsed'] = (train['week'].astype(int)-1)*7 + train['week_day'].astype(int)
val['days_elapsed'] = (val['week'].astype(int)-1)*7 + val['week_day'].astype(int)


# Data Merge

In [12]:
BASE_PATH = "../preprocessed_data/"
merge_paths = list(set(glob(BASE_PATH+"*.csv")) - \
                   set([BASE_PATH+"session_wise_product_attribute_frequency_agg_features.csv", \
                        BASE_PATH+"product_click_features_by_session.csv", \
                        BASE_PATH+"product_material_attribute_one-hot.csv"]))



In [13]:
len(merge_paths)

22

In [14]:
train, val = merge_data(train, val, merge_paths)


Train:  (3507990, 93) Val:  (687192, 92)


In [15]:
train.memory_usage(deep=True).sum()/(1024*1024*1024), val.memory_usage(deep=True).sum()/(1024*1024*1024)

(7.410801725462079, 1.4713629335165024)

In [16]:
train, val = merge_data(train, val, [BASE_PATH+"session_wise_product_attribute_frequency_agg_features.csv", \
                                     BASE_PATH+"product_click_features_by_session.csv"], merge_cols=["session_id", "product_id"], flag=1)



Train:  (3507990, 112) Val:  (687192, 111)


In [17]:
train["days_elapsed_since_first_user_action"] = train["days_elapsed"] - train["first_user_action_days_elapsed"]
val["days_elapsed_since_first_user_action"] = val["days_elapsed"] - val["first_user_action_days_elapsed"]


In [18]:
train["diff_prod_price_from_user_tier_mean"] = train["product_price"] - train["user_tier_mean_product_price"]
train["diff_prod_start_online_date_from_user_tier_mean"] = train["product_price"] - train["user_tier_mean_product_start_online_date"]

val["diff_prod_price_from_user_tier_mean"] = val["product_price"] - val["user_tier_mean_product_price"]
val["diff_prod_start_online_date_from_user_tier_mean"] = val["product_price"] - val["user_tier_mean_product_start_online_date"]


In [19]:
train["diff_start_online_date_from_user_tier_clicked_mean"] = train["start_online_date"] - train["user_tier_mean_click_product_price"]
train["diff_start_online_date_from_user_tier_clicked_mean"] = train["start_online_date"] - train["user_tier_mean_click_product_start_online_date"]

val["diff_start_online_date_from_user_tier_clicked_mean"] = val["start_online_date"] - val["user_tier_mean_click_product_price"]
val["diff_start_online_date_from_user_tier_clicked_mean"] = val["start_online_date"] - val["user_tier_mean_click_product_start_online_date"]


In [20]:
train["diff_prod_price_from_session_mean"] = train["product_price"] - train["mean_session_product_price"]
train["diff_prod_price_from_query_mean"] = train["product_price"] - train["max_query_price"]

val["diff_prod_price_from_session_mean"] = val["product_price"] - val["mean_session_product_price"]
val["diff_prod_price_from_query_mean"] = val["product_price"] - val["max_query_price"]


In [21]:
dic = json.load(open("../preprocessed_data/average_click_values.json", "r"))


In [22]:
dic

{'mean_click_price': 0.0017881393432617188,
 'mean_click_start_online_date': 414.747518851918}

In [23]:
train["diff_prod_price_from_click_mean"] = train["product_price"] - dic["mean_click_price"]
val["diff_prod_price_from_click_mean"] = val["product_price"] - dic["mean_click_price"]


In [24]:
train["diff_start_online_date_from_session_mean"] = train["start_online_date"] - train["session_start_online_date_mean"]
train["diff_start_online_date_from_query_mean"] = train["start_online_date"] - train["mean_query_start_online_date"]

val["diff_start_online_date_from_session_mean"] = val["start_online_date"] - val["session_start_online_date_mean"]
val["diff_start_online_date_from_query_mean"] = val["start_online_date"] - val["mean_query_start_online_date"]


In [25]:
train["diff_start_online_date_from_click_mean"] = train["start_online_date"] - dic["mean_click_start_online_date"]
val["diff_start_online_date_from_click_mean"] = val["start_online_date"] - dic["mean_click_start_online_date"]


In [26]:
del attribute_df
gc.collect()


66

In [27]:
train_rank_features = pd.read_csv("../preprocessed_data/rank_features/train_rank_features_and_query_prod_list.csv")


In [28]:
val_rank_features = pd.read_csv("../preprocessed_data/rank_features/val_rank_features_and_query_prod_list.csv")


In [29]:
train_rank_features = train_rank_features[['query_id', 'user_id', 'session_id', 'product_id', 'price_rank', \
                                           'start_online_date_rank', 'user_step']]
val_rank_features = val_rank_features[['query_id', 'user_id', 'session_id', 'product_id', 'price_rank', \
                                           'start_online_date_rank', 'user_step']]


In [30]:
train = train.merge(train_rank_features, on=["query_id", "user_id", "session_id", "product_id"], how="left")
val = val.merge(val_rank_features, on=["query_id", "user_id", "session_id", "product_id"], how="left")


In [31]:
train.shape, val.shape

((3507990, 125), (687192, 124))

In [32]:
set(train.columns)-set(val.columns)

{'is_click'}

In [33]:
train.memory_usage(deep=True).sum()/(1024*1024*1024), val.memory_usage(deep=True).sum()/(1024*1024*1024)


(8.546569777652621, 1.685558583587408)

In [34]:
BASE_PATH = "../preprocessed_data/clickout_features/"

In [35]:
merge_paths = glob(BASE_PATH+"*.csv")
done_files = []


In [36]:
train, val = merge_data(train, val, merge_paths)


Train:  (3507990, 175) Val:  (687192, 174)


In [37]:
is_click = train.is_click.values
train = train.drop(["is_click"], 1)
train["is_click"] = is_click

In [38]:
train.shape, val.shape

((3507990, 175), (687192, 174))

In [39]:
train.memory_usage(deep=True).sum()//(1024*1024*1024), val.memory_usage(deep=True).sum()//(1024*1024*1024)

(10, 2)